### Análise de falhas em amostra de voos "inconsistentes" de uma aeronave

O notebook apresenta uma análise relativa ao disparo de falhas no Sistema de Bleed de uma aeronave. Na análise abaixo, somente foram considerados arquivos cuja aeronave apresentou uma única fase de voo. Isso significa que alguns dos arquivos de voo de uma aeronave são inconsistentes, pois não apresentam a conclusão das fases normais de um voo. Neste caso, os arquivos apresentam informações de voos que são apenas de subida da aeronave. O objetivo é identificar se existe alguma relação entre a fase de voo inconsistente e o disparo de falhas no Sistema de Bleed, visto que, inicialmente, as falhas parecem estar concentradas em arquivos cuja aeronave apresenta uma única fase de voo.

In [128]:
import pandas as pd
import os

In [110]:
df = None
folder = "./data"
archives = [arq for arq in os.listdir(folder) if arq.endswith('.parquet')]
columns = ['message0418DAA-1', 'message0422DAA-1', 'phaseOfFlight-1']
# dtypes = {'message0418DAA-1': 'int8', 'message0422DAA-1': 'int8'}

Abaixo, encontra-se a lógica utilizada para filtrar os arquivos que compõem o novo dataframe. São considerados os seguintes atributos: as variáveis-alvo ('message0418DAA-1' e 'message0418DAA-2') e a fase de voo ('flight_phase').

Para realizar uma filtragem que considera apenas arquivos com inconsistência na fase de voo, o loop verifica a soma de valores distintos no atributo 'phaseOfFlight-1'. Caso a soma seja igual a 1, significa que o arquivo apresenta apenas uma fase de voo. Neste caso, o arquivo é adicionado ao dataframe.

In [111]:
for archive in archives:
    voo_normal = True
    path_archive = os.path.join(folder, archive)
    df_temp = pd.read_parquet(path_archive, columns=columns)
    df_temp['flight'] = archive
    df_temp['message0418DAA-1'] = df_temp['message0418DAA-1'].astype('Int8')
    df_temp['message0422DAA-1'] = df_temp['message0422DAA-1'].astype('Int8')
    df_temp['phaseOfFlight-1'] = df_temp['phaseOfFlight-1'].astype('Int8')
    df_temp.fillna(method='ffill', inplace=True)
    df_temp.fillna(method='bfill', inplace=True)
    estados_voo = df_temp['phaseOfFlight-1'].unique()
    if len(estados_voo) == 1:
        if df is None:
            df = df_temp
        else:
            df = pd.concat([df, df_temp])
    df_temp = None
    estados_voo = None

In [112]:
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
df

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
0,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
1,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
2,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
3,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
4,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
...,...,...,...,...
14536,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14537,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14538,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet
14539,0,0,0,TCRF_ARCHIVE_06120018_20230718132109.parquet


In [125]:
df["flight"].nunique()

163

In [115]:
df["phaseOfFlight-1"].value_counts()

phaseOfFlight-1
0    2833869
Name: count, dtype: Int64

In [116]:
df.head()

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
0,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
1,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
2,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
3,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet
4,0,0,0,TCRF_ARCHIVE_06120018_20220524155501.parquet


In [117]:
df.isna().sum()

message0418DAA-1    0
message0422DAA-1    0
phaseOfFlight-1     0
flight              0
dtype: int64

In [126]:
df["message0422DAA-1"].value_counts()

message0422DAA-1
0    2747896
7      85793
5        180
Name: count, dtype: Int64

In [127]:
df["message0418DAA-1"].value_counts()

message0418DAA-1
0    2830928
7       2861
5         80
Name: count, dtype: Int64

In [120]:
df[df['message0422DAA-1'] == 7]

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
2723,0,7,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2724,0,7,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2725,0,7,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2726,0,7,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2727,0,7,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
...,...,...,...,...
31836,0,7,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
31837,0,7,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
31838,0,7,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
31839,0,7,0,TCRF_ARCHIVE_06120018_20230602190605.parquet


In [121]:
df[df['message0422DAA-1'] == 5]

,message0418DAA-1,message0422DAA-1,phaseOfFlight-1,flight
2663,0,5,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2664,0,5,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2665,0,5,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2666,0,5,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
2667,0,5,0,TCRF_ARCHIVE_06120018_20230601141301.parquet
...,...,...,...,...
5258,0,5,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
5259,0,5,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
5260,0,5,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
5261,0,5,0,TCRF_ARCHIVE_06120018_20230602190605.parquet
